In [1]:
import torch.nn as nn
import os
import sys
import glob
import shutil
import codecs
import subprocess

import pandas as pd
import numpy as np
import time

# %pylab inline
from PIL import Image
#train_df = pd.read_csv('../input/train.csv')

# from multiprocessing.pool import ThreadPool
import pretrainedmodels

In [2]:
class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x

In [2]:
# 读取query 每秒帧 并进行排序
query_imgs_path =glob.glob('./query_frame/*/*.jpg')
# for id in pd.read_csv('../input/submit_example.csv')['query_id']:
#     query_imgs_path += glob.glob('../input/query_frame/' + id + '/*.jpg')
query_imgs_path.sort(key=lambda x: x.lower())

# print(query_imgs_path)

In [3]:
# 读取refer 每帧帧 并进行排序

refer_imgs_path = glob.glob('./refer_frame/*/*.jpg')
refer_imgs_path.sort(key=lambda x: x.lower())
# print(refer_imgs_path)

In [4]:
# import cv2
#import imagehash
from PIL import Image
from tqdm import tqdm_notebook
from scipy.spatial.distance import cosine

def compute_cosin_distance(Q, feats, names):
    """
    feats and Q: L2-normalize, n*d
    """
    dists = np.dot(Q, feats.T)
#     print(dists)
    idxs = np.argsort(dists)[::-1]
#     print(idxs)
    rank_dists = dists[idxs]
#     print(rank_dists)
    rank_names = [names[k] for k in idxs]
#     print(rank_names)
    return idxs, rank_dists, rank_names

def hamming_distance(chaine1, chaine2):
    return sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2))

# 将图片经过去除最后一层的预训练的模型的特征进行比对

In [5]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
class QRDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
#         print(self.img_path)
        self.img_label = np.zeros(len(img_path))
    
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, self.img_path[index]

    def __len__(self):
        return len(self.img_path)
    
# model = models.resnet18(True).cuda()
# # print(model)
# del model.fc
# model.fc=lambda x:x

# model = se_resnet50(num_classes=1000,pretrained=True)
# #print(model)
# #model.load_state_dict(torch.load('seresnet50.pkl'))
# del model.fc
# model=lambda x:x
# model.eval()
# model = torch.load('/home/zhuyuan/桌面/4_10000model.pth')
# del model.Linear_layer
# model.Linear_layer=lambda x:x
# device = torch.device('cuda')  # 若能使用cuda，则使用cuda
# model = model.to(device)


model_name = 'se_resnet50' # could be fbresnet152 or inceptionresnetv2           # 导入senet50模型
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.eval()
# print(model)
del model.last_linear    #删去最后一层
model.last_linear=lambda x:x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)

def predit_file(path):
    if not isinstance(path, list):
        path = [path]
    
    # print(path)
    
    data_loader = torch.utils.data.DataLoader(
        QRDataset(path, 
                transforms.Compose([
#                             transforms.Resize((256, 256)),  # 先调整图片大小至256x256
#                             transforms.RandomCrop((224, 224)),
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=40, shuffle=False, num_workers=0,
    )
    
    img_feat = []
    with torch.no_grad():
        for batch_data in tqdm_notebook(data_loader):
            batch_x, batch_y = batch_data
#             print(batch_x)
#             print(batch_y)
            
            # print(batch_y[:10])
            batch_x = Variable(batch_x).cuda()
            
            batch_x = batch_x.to(batch_x)
#             device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            feat_pred = model(batch_x)
#             print(feat_pred.shape)

            # max-pooling
            # feat_pred = F.max_pool2d(feat_pred, kernel_size=(24, 32))
            
            # ave-pooling
            # feat_pred = F.avg_pool2d(feat_pred, kernel_size=(24, 32))[:, :, 0, 0]
            
            #print(feat_pred.shape, batch_x.shape)
            feat_pred = feat_pred.data.cpu().numpy()
            # feat_pred = feat_pred.max(-1).max(-1)
            
            # feat_pred = feat_pred.reshape((-1, 512))
            img_feat.append(feat_pred)
            
            del feat_pred
            # img_feat.append(feat_pred)
            
    img_feat = np.vstack(img_feat)
    return img_feat

In [6]:
query_cnn = predit_file(list(query_imgs_path[:]))

In [8]:
refer_cnn = predit_file(list(refer_imgs_path[:]))

In [9]:
# -*- coding: utf-8 -*-
import os, sys, codecs
import glob

import numpy as np
# import cv2

from sklearn.preprocessing import normalize as sknormalize
from sklearn.decomposition import PCA

def normalize(x, copy=False):
    """
    A helper function that wraps the function of the same name in sklearn.
    This helper handles the case of a single column vector.
    """
    if type(x) == np.ndarray and len(x.shape) == 1:
        return np.squeeze(sknormalize(x.reshape(1,-1), copy=copy))
        #return np.squeeze(x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis])
    else:
        return sknormalize(x, copy=copy)
        #return x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis]

# 特征归一化
query_cnn = normalize(query_cnn)
refer_cnn = normalize(refer_cnn)

In [10]:
print(query_cnn.shape)

(2228, 512)


##  调用faiss库进行加速     此次比赛中,此操作将最后匹配时间缩减了大概10倍左右

In [14]:
import faiss


import numpy as np

d=int(query_cnn.shape[1])
nlist = 5                      #聚类中心的个数
k = 5
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

# assert not index.is_trained
index.train(refer_cnn)
# assert index.is_trained

index.add(refer_cnn)                  # add may be a bit slower as well
D1, I1 = index.search(query_cnn, k)     # actual search

index.nprobe = 10              # default nprobe is 1, try a few more
D2, I2 = index.search(query_cnn, k)

# 输出相匹配的图像名称,以及其相似度

In [15]:
num=query_cnn.shape[0]
h=0
with open('senet50.txt','w') as f:
    for i in range(num):
        for id,j in enumerate(I1[i]):
            if(D1[i][id]>0.8):
                print(query_imgs_path[i].split('/')[-1], refer_imgs_path[j].split('/')[-1],D1[i][id],file=f)
                print(query_imgs_path[i].split('/')[-1], refer_imgs_path[j].split('/')[-1],D1[i][id])

21443adc-b911-11e9-ad99-fa163ee49799_00001.jpg 1629260900_05192.jpg 0.8356558
21443adc-b911-11e9-ad99-fa163ee49799_00001.jpg 1376903200_03872.jpg 0.8321612
21443adc-b911-11e9-ad99-fa163ee49799_00001.jpg 1376903200_03866.jpg 0.830517
21443adc-b911-11e9-ad99-fa163ee49799_00001.jpg 1629260900_04272.jpg 0.829913
21443adc-b911-11e9-ad99-fa163ee49799_00001.jpg 1629260900_04281.jpg 0.82938623
21443adc-b911-11e9-ad99-fa163ee49799_00002.jpg 1753812600_04407.jpg 0.838609
21443adc-b911-11e9-ad99-fa163ee49799_00002.jpg 1376903200_03872.jpg 0.83653665
21443adc-b911-11e9-ad99-fa163ee49799_00002.jpg 1376903200_03868.jpg 0.83388853
21443adc-b911-11e9-ad99-fa163ee49799_00002.jpg 1376903200_03866.jpg 0.8328382
21443adc-b911-11e9-ad99-fa163ee49799_00002.jpg 1629260900_05192.jpg 0.8321568
21443adc-b911-11e9-ad99-fa163ee49799_00004.jpg 1804312000_05950.jpg 0.80018854
21443adc-b911-11e9-ad99-fa163ee49799_00006.jpg 1629260900_04174.jpg 0.8056561
21443adc-b911-11e9-ad99-fa163ee49799_00008.jpg 1629260900_04169

2a2645c4-b942-11e9-8b81-fa163ee49799_00024.jpg 3043930400_02806.jpg 0.812456
2a2645c4-b942-11e9-8b81-fa163ee49799_00024.jpg 3043930400_06946.jpg 0.8096652
2a2645c4-b942-11e9-8b81-fa163ee49799_00024.jpg 3043930400_02805.jpg 0.8026707
2a2645c4-b942-11e9-8b81-fa163ee49799_00025.jpg 2620315400_02740.jpg 0.8386098
2a2645c4-b942-11e9-8b81-fa163ee49799_00025.jpg 2927857700_05825.jpg 0.8283999
2a2645c4-b942-11e9-8b81-fa163ee49799_00025.jpg 2927857700_05826.jpg 0.8206117
2a2645c4-b942-11e9-8b81-fa163ee49799_00025.jpg 1398481500_03204.jpg 0.8194156
2a2645c4-b942-11e9-8b81-fa163ee49799_00025.jpg 3043930400_04836.jpg 0.8158647
2a2645c4-b942-11e9-8b81-fa163ee49799_00026.jpg 3043930400_06947.jpg 0.89452887
2a2645c4-b942-11e9-8b81-fa163ee49799_00026.jpg 2620315400_02740.jpg 0.8485442
2a2645c4-b942-11e9-8b81-fa163ee49799_00026.jpg 2620315400_05228.jpg 0.8421079
2a2645c4-b942-11e9-8b81-fa163ee49799_00026.jpg 2927857700_05826.jpg 0.83478546
2a2645c4-b942-11e9-8b81-fa163ee49799_00026.jpg 3043930400_04836

374a3a0c-b93e-11e9-8b81-fa163ee49799_00075.jpg 2927857700_06347.jpg 0.8517684
374a3a0c-b93e-11e9-8b81-fa163ee49799_00075.jpg 2927857700_04305.jpg 0.83851326
374a3a0c-b93e-11e9-8b81-fa163ee49799_00075.jpg 2927857700_06353.jpg 0.83719903
374a3a0c-b93e-11e9-8b81-fa163ee49799_00075.jpg 2927857700_06348.jpg 0.8369093
374a3a0c-b93e-11e9-8b81-fa163ee49799_00076.jpg 2333805400_02892.jpg 0.8136933
374a3a0c-b93e-11e9-8b81-fa163ee49799_00076.jpg 3043930400_04249.jpg 0.80852175
374a3a0c-b93e-11e9-8b81-fa163ee49799_00076.jpg 2927857700_07688.jpg 0.8076939
374a3a0c-b93e-11e9-8b81-fa163ee49799_00076.jpg 2620315400_00575.jpg 0.8021203
374a3a0c-b93e-11e9-8b81-fa163ee49799_00076.jpg 2333805400_02885.jpg 0.8011291
374a3a0c-b93e-11e9-8b81-fa163ee49799_00078.jpg 3043930400_03345.jpg 0.8563641
374a3a0c-b93e-11e9-8b81-fa163ee49799_00078.jpg 3043930400_03344.jpg 0.85543627
374a3a0c-b93e-11e9-8b81-fa163ee49799_00078.jpg 3043930400_03347.jpg 0.8518224
374a3a0c-b93e-11e9-8b81-fa163ee49799_00078.jpg 3043930400_03

68848f02-b996-11e9-8b81-fa163ee49799_00012.jpg 1398481500_03358.jpg 0.9323269
68848f02-b996-11e9-8b81-fa163ee49799_00012.jpg 1398481500_01856.jpg 0.8397403
68848f02-b996-11e9-8b81-fa163ee49799_00012.jpg 1629260900_03955.jpg 0.8393959
68848f02-b996-11e9-8b81-fa163ee49799_00012.jpg 1629260900_04389.jpg 0.83196276
68848f02-b996-11e9-8b81-fa163ee49799_00012.jpg 1629260900_02910.jpg 0.8318907
68848f02-b996-11e9-8b81-fa163ee49799_00013.jpg 1530507600_04610.jpg 0.8381472
68848f02-b996-11e9-8b81-fa163ee49799_00013.jpg 1530507600_04611.jpg 0.83491087
68848f02-b996-11e9-8b81-fa163ee49799_00013.jpg 1376903200_04268.jpg 0.82188857
68848f02-b996-11e9-8b81-fa163ee49799_00013.jpg 1376903200_04269.jpg 0.8218584
68848f02-b996-11e9-8b81-fa163ee49799_00013.jpg 1398481500_03360.jpg 0.81671864
68848f02-b996-11e9-8b81-fa163ee49799_00015.jpg 1398481500_03361.jpg 0.940858
68848f02-b996-11e9-8b81-fa163ee49799_00015.jpg 1398481500_03362.jpg 0.8842851
68848f02-b996-11e9-8b81-fa163ee49799_00015.jpg 1398481500_033

984a4d86-b99f-11e9-8b81-fa163ee49799_00020.jpg 1530507600_04030.jpg 0.81640977
984a4d86-b99f-11e9-8b81-fa163ee49799_00021.jpg 1530507600_03604.jpg 0.8263962
984a4d86-b99f-11e9-8b81-fa163ee49799_00021.jpg 1596046700_00295.jpg 0.81192374
984a4d86-b99f-11e9-8b81-fa163ee49799_00021.jpg 1398481500_02801.jpg 0.80734384
984a4d86-b99f-11e9-8b81-fa163ee49799_00021.jpg 1629260900_01707.jpg 0.80633307
984a4d86-b99f-11e9-8b81-fa163ee49799_00021.jpg 1530507600_03664.jpg 0.80111146
984a4d86-b99f-11e9-8b81-fa163ee49799_00022.jpg 1530507600_03673.jpg 0.8528421
984a4d86-b99f-11e9-8b81-fa163ee49799_00022.jpg 1530507600_03694.jpg 0.84911424
984a4d86-b99f-11e9-8b81-fa163ee49799_00022.jpg 1530507600_03671.jpg 0.8479266
984a4d86-b99f-11e9-8b81-fa163ee49799_00022.jpg 1530507600_03670.jpg 0.8452561
984a4d86-b99f-11e9-8b81-fa163ee49799_00022.jpg 1530507600_03616.jpg 0.84494567
984a4d86-b99f-11e9-8b81-fa163ee49799_00023.jpg 1530507600_03671.jpg 0.8547041
984a4d86-b99f-11e9-8b81-fa163ee49799_00023.jpg 1530507600

c008e798-b98f-11e9-8b81-fa163ee49799_00085.jpg 2333805400_00287.jpg 0.8140503
c008e798-b98f-11e9-8b81-fa163ee49799_00085.jpg 1753812600_01522.jpg 0.8098715
c008e798-b98f-11e9-8b81-fa163ee49799_00087.jpg 2927857700_06863.jpg 0.81033623
c008e798-b98f-11e9-8b81-fa163ee49799_00087.jpg 1782169900_06131.jpg 0.8052975
c008e798-b98f-11e9-8b81-fa163ee49799_00096.jpg 1629260900_04179.jpg 0.80115145
c008e798-b98f-11e9-8b81-fa163ee49799_00101.jpg 1629260900_04184.jpg 0.80586094
c008e798-b98f-11e9-8b81-fa163ee49799_00101.jpg 1376903200_03739.jpg 0.8026631
c008e798-b98f-11e9-8b81-fa163ee49799_00101.jpg 1629260900_04191.jpg 0.801849
c008e798-b98f-11e9-8b81-fa163ee49799_00102.jpg 2333805400_01473.jpg 0.8042471
c008e798-b98f-11e9-8b81-fa163ee49799_00102.jpg 1629260900_04230.jpg 0.80358696
c008e798-b98f-11e9-8b81-fa163ee49799_00102.jpg 2333805400_01474.jpg 0.80259717
c008e798-b98f-11e9-8b81-fa163ee49799_00102.jpg 1629260900_04184.jpg 0.8000714
c008e798-b98f-11e9-8b81-fa163ee49799_00104.jpg 1629260900_04

da5ab27a-b98f-11e9-8b81-fa163ee49799_00158.jpg 2620315400_03207.jpg 0.8037428
da5ab27a-b98f-11e9-8b81-fa163ee49799_00158.jpg 2620315400_03204.jpg 0.8018849
da5ab27a-b98f-11e9-8b81-fa163ee49799_00161.jpg 1629260900_04389.jpg 0.8289137
da5ab27a-b98f-11e9-8b81-fa163ee49799_00161.jpg 1629260900_04390.jpg 0.82138175
da5ab27a-b98f-11e9-8b81-fa163ee49799_00161.jpg 1629260900_04457.jpg 0.8177273
da5ab27a-b98f-11e9-8b81-fa163ee49799_00161.jpg 1629260900_03954.jpg 0.81266665
da5ab27a-b98f-11e9-8b81-fa163ee49799_00161.jpg 1629260900_04516.jpg 0.8122381
da5ab27a-b98f-11e9-8b81-fa163ee49799_00162.jpg 1376903200_00978.jpg 0.84015226
da5ab27a-b98f-11e9-8b81-fa163ee49799_00162.jpg 1376903200_03717.jpg 0.8365118
da5ab27a-b98f-11e9-8b81-fa163ee49799_00162.jpg 1398481500_01957.jpg 0.83537674
da5ab27a-b98f-11e9-8b81-fa163ee49799_00162.jpg 2400411900_05170.jpg 0.83394426
da5ab27a-b98f-11e9-8b81-fa163ee49799_00162.jpg 1629260900_04114.jpg 0.8321954
da5ab27a-b98f-11e9-8b81-fa163ee49799_00163.jpg 1530507600_0

# 下一步对数据进行去噪,进行时间对齐